# ===========================
# NHẬN DẠNG 3 HÌNH: TRÒN, VUÔNG, TAM GIÁC
# ===========================
# Model này để nhận dạng chính xác hơn với ảnh nhiều màu và nhiều hình


In [1]:
# ===========================
# HÀM TIỀN XỬ LÝ + DETECT CONTOUR BỔ SUNG
# ===========================
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import gradio as gr

def convert_to_grayscale(img):
    if img is None:
        return None
    if len(img.shape) == 2:
        return img
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def detect_shapes_improved(image_path_or_array, min_area=100, use_grayscale=True):
    
    # 1. Đọc ảnh (path hoặc array)
    if isinstance(image_path_or_array, str):
        img = cv2.imread(image_path_or_array)
        if img is None:
            raise FileNotFoundError(f"Không đọc được ảnh: {image_path_or_array}")
    else:
        img = image_path_or_array.copy()

    # 2. Chuyển grayscale
    if use_grayscale:
        gray = convert_to_grayscale(img)
    else:
        # vẫn nên có grayscale để threshold
        gray = convert_to_grayscale(img)

    # 3. Giảm nhiễu
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # 4. Threshold (adaptive + fallback Otsu)
    try:
        mask = cv2.adaptiveThreshold(
            blurred, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,
            11, 2
        )
        if np.count_nonzero(mask) < 10:
            _, mask = cv2.threshold(
                blurred, 0, 255,
                cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
            )
    except Exception:
        _, mask = cv2.threshold(
            blurred, 0, 255,
            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
        )

    # 5. Morphology để sạch nhiễu
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)

    # 6. Tìm contour
    contours_info = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours_info[0] if len(contours_info) == 2 else contours_info[1]

    # 7. Lọc theo min_area
    filtered = [c for c in contours if cv2.contourArea(c) >= max(1, min_area)]

    return filtered, img, mask


# ===========================
# HÀM NHẬN DẠNG CẢI THIỆN VỚI PREPROCESSING 
# ===========================
def detect_and_classify_shapes_improved(image_path_or_array, model, img_size=None, min_area=100, use_grayscale=True, confidence_threshold=0.5):
    # Detect model input shape
    model_input_shape = model.input_shape[1:]  # Bỏ qua batch dimension
    if len(model_input_shape) == 3:
        # Format: (height, width, channels)
        model_img_size = model_input_shape[0]
        model_channels = model_input_shape[2]
    elif len(model_input_shape) == 2:
        # Format: (height, width) - grayscale
        model_img_size = model_input_shape[0]
        model_channels = 1
    else:
        # Fallback
        model_img_size = 200 
        model_channels = 1   # Grayscale
    
    if img_size is None:
        img_size = model_img_size

    if model_channels == 1:
        use_grayscale = True
        print("Model yêu cầu ảnh grayscale, tự động chuyển đổi...")
    
    print(f"Model input: {img_size}x{img_size}x{model_channels}")

    
    contours, img, mask = detect_shapes_improved(
        image_path_or_array,
        min_area=min_area,
        use_grayscale=use_grayscale
    )
    print(f"Đã tìm thấy {len(contours)} contours")
    
    # Lấy ảnh grayscale để dùng cho model prediction nếu model yêu cầu
    if model_channels == 1:
        img_gray = convert_to_grayscale(img)
    else:
        img_gray = None
    
    result_img = img.copy()
    
    # Tự động detect số classes từ model output shape
    try:
        dummy_input = np.zeros((1, img_size, img_size, model_channels), dtype='float32')
        dummy_pred = model.predict(dummy_input, verbose=0)
        num_classes = dummy_pred.shape[1] if len(dummy_pred.shape) > 1 else len(dummy_pred[0])
    except:
        num_classes = 3  # fallback
    
    # Labels dựa trên số classes
    if num_classes == 3:
        labels = ['circle', 'square', 'triangle']
        labels_vn = ['Hinh tron', 'Hinh vuong', 'Tam giac']
    else:
        labels = [f'class_{i}' for i in range(num_classes)]
        labels_vn = [f'Lớp {i}' for i in range(num_classes)]
    
    print(f"Model có {num_classes} classes: {labels}")
    print(f"Ngưỡng confidence: {confidence_threshold:.2f}, Min area: {min_area}")
    
    detections = []
    total_predictions = 0
    low_confidence_count = 0
    
    # Duyệt qua contours
    for contour in contours:
        # Lấy bounding box với padding
        x, y, w, h = cv2.boundingRect(contour)
        padding = max(5, int(min(w, h) * 0.1))  # Padding động
        
        x = max(0, x - padding)
        y = max(0, y - padding)
        w = min(img.shape[1] - x, w + 2 * padding)
        h = min(img.shape[0] - y, h + 2 * padding)
        
        # Crop ROI - sử dụng ảnh grayscale nếu model yêu cầu
        if model_channels == 1 and img_gray is not None:
            roi_gray_crop = img_gray[y:y+h, x:x+w]
            if roi_gray_crop.size == 0:
                continue
            roi_resized = cv2.resize(roi_gray_crop, (img_size, img_size))
            roi_normalized = roi_resized.reshape(1, img_size, img_size, 1).astype('float32') / 255.0
        else:
            roi_bgr = img[y:y+h, x:x+w]
            if roi_bgr.size == 0:
                continue
            if model_channels == 3:
                roi_rgb = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2RGB)
                roi_resized = cv2.resize(roi_rgb, (img_size, img_size))
                roi_normalized = roi_resized.reshape(1, img_size, img_size, 3).astype('float32') / 255.0
            else:
                roi_gray = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2GRAY)
                roi_resized = cv2.resize(roi_gray, (img_size, img_size))
                roi_normalized = roi_resized.reshape(1, img_size, img_size, 1).astype('float32') / 255.0
        
        # Dự đoán
        prediction = model.predict(roi_normalized, verbose=0)
        
        if len(prediction.shape) == 2:
            pred_array = prediction[0]
        else:
            pred_array = prediction[0] if len(prediction) > 0 else prediction
        
        class_idx = np.argmax(pred_array)
        confidence = float(pred_array[class_idx])
        total_predictions += 1
        
        if class_idx >= len(labels):
            print(f"Warning: class_idx {class_idx} >= số classes {len(labels)}, bỏ qua")
            continue
        
        if confidence > confidence_threshold:
            colors = {
                0: (0, 255, 0),    # Circle - Xanh lá
                1: (255, 0, 0),    # Square - Đỏ
                2: (0, 0, 255)     # Triangle - Xanh dương
            }
            color = colors.get(class_idx, (128, 128, 128))
            
            cv2.rectangle(result_img, (x, y), (x + w, y + h), color, 3)
            cv2.drawContours(result_img, [contour], -1, color, 2)
            
            label_text = f"{labels_vn[class_idx]} {confidence:.2f}"
            (text_width, text_height), baseline = cv2.getTextSize(
                label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2
            )
            
            cv2.rectangle(
                result_img,
                (x, y - text_height - 15),
                (x + text_width + 10, y),
                color,
                -1
            )
            
            cv2.putText(
                result_img,
                label_text,
                (x + 5, y - 8),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (255, 255, 255),
                2
            )
            
            detections.append({
                'x': int(x),
                'y': int(y),
                'w': int(w),
                'h': int(h),
                'class': labels[class_idx],
                'class_vn': labels_vn[class_idx],
                'confidence': float(confidence)
            })
        else:
            low_confidence_count += 1
            if confidence > 0.3:
                print(f"Hình bị bỏ qua: {labels[class_idx]} (confidence: {confidence:.2%} < {confidence_threshold:.2%})")
    
    print(f"\nTổng kết: {total_predictions} hình được dự đoán, {len(detections)} hình được chấp nhận (confidence > {confidence_threshold:.2%})")
    if low_confidence_count > 0:
        print(f"   {low_confidence_count} hình bị bỏ qua do confidence thấp")
    
    return result_img, detections

print("Hàm detect_and_classify_shapes_improved!")


# ===========================
# TẠO GIAO DIỆN GRADIO CẢI THIỆN
# ===========================
_loaded_model = None

def load_model_for_gradio(model_path=None):
    global _loaded_model
    
    if _loaded_model is not None:
        return _loaded_model
    
    default_models = [
        "shapes_3class_improved.h5"
    ]
    
    if model_path is None:
        for model_file in default_models:
            try:
                _loaded_model = load_model(model_file)
                print(f"✅ Đã load model: {model_file}")
                return _loaded_model
            except:
                continue
        raise FileNotFoundError("Không tìm thấy model nào. Vui lòng chỉ định đường dẫn model.")
    else:
        _loaded_model = load_model(model_path)
        print(f"✅ Đã load model: {model_path}")
        return _loaded_model

def predict_shape(image, model_path=None, use_grayscale=True):
    global _loaded_model
    
    if image is None:
        return None, "Vui lòng upload ảnh"
    
    try:
        if _loaded_model is None:
            load_model_for_gradio(model_path)
        
        # Gradio cho RGB → chuyển BGR
        if len(image.shape) == 3:
            image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        else:
            image_bgr = image
        
        result_img, detections = detect_and_classify_shapes_improved(
            image_bgr, 
            _loaded_model, 
            min_area=50,
            use_grayscale=use_grayscale,
            confidence_threshold=0.4
        )
        
        result_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
        
        info_text = f"Đã phát hiện {len(detections)} hình:\n"
        for i, det in enumerate(detections, 1):
            info_text += f"{i}. {det['class_vn']} (độ tin cậy: {det['confidence']:.2%})\n"
        
        if len(detections) == 0:
            info_text = "Không phát hiện hình nào (độ tin cậy < 40%)"
        
        return result_rgb, info_text
        
    except Exception as e:
        error_msg = f"Lỗi: {str(e)}"
        print(error_msg)
        return None, error_msg

def create_improved_gradio_interface(model_path=None):
    try:
        load_model_for_gradio(model_path)
    except Exception as e:
        print(f"Cảnh báo: {e}")
        print("Bạn có thể load model sau khi chạy interface")
    
    with gr.Blocks(title="Nhận dạng hình dạng cải thiện") as demo:
        gr.Markdown(
            """
            # Nhận dạng Hình Dạng
            ### Upload ảnh để nhận dạng các hình: Tròn, Vuông, Tam giác
            """
        )
        
        with gr.Row():
            with gr.Column():
                image_input = gr.Image(
                    label="Upload ảnh",
                    type="numpy",
                    height=400
                )
                    
                predict_btn = gr.Button("Nhận dạng", variant="primary")
            
            with gr.Column():
                image_output = gr.Image(
                    label="Kết quả nhận dạng",
                    type="numpy",
                    height=400
                )
                info_output = gr.Textbox(
                    label="Thông tin",
                    lines=10,
                    interactive=False
                )
        
        predict_btn.click(
            fn=lambda img: predict_shape(img, None, True),
            inputs=[image_input],
            outputs=[image_output, info_output]
        )
        
        image_input.change(
            fn=lambda img: predict_shape(img, None, True),
            inputs=[image_input],
            outputs=[image_output, info_output]
        )
    
    return demo

print("Hàm create_improved_gradio_interface()!")
print("Mặc định, hệ thống sẽ tự động chuyển ảnh sang grayscale (ảnh đen trắng) để xử lý!")


Hàm detect_and_classify_shapes_improved!
Hàm create_improved_gradio_interface()!
Mặc định, hệ thống sẽ tự động chuyển ảnh sang grayscale (ảnh đen trắng) để xử lý!


In [2]:
# ===========================
# 1️⃣6️⃣ CHẠY GIAO DIỆN GRADIO CẢI THIỆN
# ===========================

# Tạo và launch giao diện
demo_improved = create_improved_gradio_interface()

# Launch với các tùy chọn để tránh lỗi event loop
demo_improved.launch()

print("\nGiao diện Gradio đã chạy!")


✅ Đã load model: shapes_3class_improved.h5
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.



Giao diện Gradio đã chạy!
